In [ ]:
url="http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip" 

In [ ]:

!pip install trimesh
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

     |████████████████████████████████| 635 kB 7.3 MB/s 


##Loading Data

In [ ]:
data=tf.keras.utils.get_file("modelnet.zip",url,extract=True)
DATA_DIR = os.path.join(os.path.dirname(data), "ModelNet10")

473415680/473402300 [==============================] - 11s 0us/step


In [ ]:

def parse_dataset(num_points=2048):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    class_map = {}
    folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points))
            train_labels.append(i)

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )

In [ ]:
Batch=32
num_p=2048

In [ ]:
train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(num_p)

processing class: desk
processing class: night_stand
processing class: chair
processing class: bathtub
processing class: monitor
processing class: toilet
processing class: table
processing class: sofa
processing class: dresser
processing class: bed


In [ ]:
print(CLASS_MAP)

{0: 'desk', 1: 'night_stand', 2: 'chair', 3: 'bathtub', 4: 'monitor', 5: 'toilet', 6: 'table', 7: 'sofa', 8: 'dresser', 9: 'bed'}


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))


train_dataset=train_dataset.shuffle(len(train_points))
train_dataset=train_dataset.batch(Batch)
test_dataset=test_dataset.shuffle(len(test_points))
test_dataset=test_dataset.batch(Batch)

In [ ]:
np.shape(train_dataset)

()

##Building the Model

In [ ]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [ ]:
class ortho_reg(keras.regularizers.Regularizer):
  def __init__(self, num_feat, l2_reg=0.001):
    self.num_feat=num_feat
    self.l2_reg=l2_reg
    self.eye=tf.eye(num_feat)
    
  def __call__(self,x):
    x = tf.reshape(x, (-1, self.num_feat, self.num_feat))
    xxt = tf.tensordot(x, x, axes=(2, 2))
    xxt = tf.reshape(xxt, (-1, self.num_feat, self.num_feat))
    return tf.reduce_sum(self.l2_reg * tf.square(xxt - self.eye))  

In [ ]:
lr=0.001/4

In [ ]:

def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = ortho_reg(num_features)

    x = conv_bn(inputs, 64)
    x = conv_bn(x, 128)
    x = conv_bn(x, 1024)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 512)
    x = dense_bn(x, 256)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
def pointnet():

  inputs = keras.Input(shape=(num_p, 3))
  x = tnet(inputs, 3)
  x = conv_bn(x, 64)
  x = conv_bn(x, 64)
  x = tnet(x, 64)
  x = conv_bn(x, 64)
  x = conv_bn(x, 128)
  x = conv_bn(x, 1024)
  x = layers.GlobalMaxPooling1D()(x)
  x = dense_bn(x, 512)
  x = layers.Dropout(0.3)(x)
  x = dense_bn(x, 256)
  x = layers.Dropout(0.3)(x)
  outputs = layers.Dense(10, activation="softmax")(x)
  model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
  model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr),
    metrics=["sparse_categorical_accuracy"])
  return (model,model.summary())


In [ ]:
model,sum=pointnet()

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 2048, 64)     256         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_51 (BatchNo (None, 2048, 64)     256         conv1d_33[0][0]                  
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 2048, 64)     0           batch_normalization_51[0][0]     
___________________________________________________________________________________________

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:


model.fit(train_dataset, epochs=20, validation_data=test_dataset,callbacks=[cp_callback])

Epoch 1/20
125/125 [==============================] - 45s 327ms/step - loss: 2.4509 - sparse_categorical_accuracy: 0.8837 - val_loss: 12920972466460819456.0000 - val_sparse_categorical_accuracy: 0.8051

Epoch 00001: saving model to training_3/cp.ckpt
Epoch 2/20
125/125 [==============================] - 40s 321ms/step - loss: 2.4577 - sparse_categorical_accuracy: 0.8735 - val_loss: 66479923920896.0000 - val_sparse_categorical_accuracy: 0.8128

Epoch 00002: saving model to training_3/cp.ckpt
Epoch 3/20
125/125 [==============================] - 40s 321ms/step - loss: 2.4806 - sparse_categorical_accuracy: 0.8757 - val_loss: 1514803691520.0000 - val_sparse_categorical_accuracy: 0.8150

Epoch 00003: saving model to training_3/cp.ckpt
Epoch 4/20
125/125 [==============================] - 40s 321ms/step - loss: 2.4487 - sparse_categorical_accuracy: 0.8770 - val_loss: 4749560320.0000 - val_sparse_categorical_accuracy: 0.8524

Epoch 00004: saving model to training_3/cp.ckpt
Epoch 5/20
125/125 

In [ ]:
def part_seg():
  inputs = keras.Input(shape=(num_p, 3))
  x = tnet(inputs, 3)
  x = conv_bn(x, 64)
  out1=x
  x = conv_bn(x, 128)
  out2=
  x = conv_bn(x, 128)
  out3=x
  x=tnet(x,128)
  out4=x
  x = conv_bn(x, 512)
  out5=x
  x = conv_bn(x, 2048)
  x = layers.GlobalMaxPooling1D()(x)
  ex=tf.concat([out1,out2,out3,out4,ou5,x,one_hot],axis=3)
    

